In [1]:
import imp
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization, GlobalAveragePooling2D, InputLayer, LeakyReLU
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
import os
import shutil
import glob 

/var/folders/h0/pjxkfbkd0jn4b1ngbzy6_d3c0000gn/T/ipykernel_33734/2947326146.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/Users/diganto/opt/anaconda3/envs/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
ROOT_DIR = "/Users/diganto/Data/Thesis/Brain Tumor Data Set"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
    print(dir)
    print(len(os.listdir(os.path.join(ROOT_DIR, dir))))
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

number_of_images.items()

Healthy
2087
Brain Tumor
2513


dict_items([('Healthy', 2087), ('Brain Tumor', 2513)])

In [3]:
import math


def dataFolder(p, split):
    
    if not os.path.exists("./dataset/" + p):
        os.makedirs("./dataset/" + p)
    
    for dir in os.listdir(ROOT_DIR):
        os.makedirs("./dataset/"+ p + "/" + dir)
        
        for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)), 
                                    size=(math.floor(split*number_of_images[dir])-5) ,
                                    replace = False):
            
            O = os.path.join(ROOT_DIR, dir, img) #only complete path
            D = os.path.join("./dataset/"+p, dir) #only complete path
            shutil.copy(O,D)
            #os.remove(O)
            
    else:
        print(f'{p} folder already exists')

In [13]:
#dataFolder('train', 0.7)

In [14]:
#dataFolder('test', 0.15)

In [15]:
#dataFolder('validation', 0.15)

In [4]:
from importlib.resources import path
image_data = ImageDataGenerator(rescale=1./255,validation_split = 0.2,zoom_range=(0.99,0.99),dtype=tf.float32)

def preprocessingTrain(path):
    
    image = image_data.flow_from_directory(directory=path, 
                                           target_size=(150,150), 
                                           batch_size=256, 
                                           color_mode='rgb',
                                           shuffle='True',
                                           seed=123,
                                           class_mode='binary', 
                                           subset='training',)
    
    return image

In [5]:
def preprocessingImages(path):
    
    image_data = ImageDataGenerator(rescale=1/255)
    image = image_data.flow_from_directory(directory=path,
                                           target_size = (150,150),
                                           batch_size = 32,
                                           class_mode = "binary",
                                           )
    return image

In [18]:
path = './dataset/train'
train_data= preprocessingTrain(path)

Found 2568 images belonging to 2 classes.


In [30]:
path = './dataset/validation'
val_data= preprocessingImages(path)

Found 679 images belonging to 2 classes.


In [31]:
path = './dataset/test'
test_data= preprocessingImages(path)

Found 679 images belonging to 2 classes.


In [6]:
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

model = keras.Sequential()

model.add(InputLayer(input_shape=(150,150,3)))

model.add(Conv2D(filters=32,kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=3, activation="relu", padding="same"))
model.add(MaxPool2D())


model.add(Flatten()) #


model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(1, activation="sigmoid"))


model.compile(optimizer=Adam(0.001),loss = BinaryCrossentropy(),metrics=['accuracy'])


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 87616)             0         
                                                                 
 dense (Dense)               (None, 128)               1

2022-11-17 00:01:23.944557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True,
    show_layer_names=True,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:
%pip install pydot

In [26]:
# Early stopping and model checkpoint
from keras import utils, callbacks

earlystopping = callbacks.EarlyStopping(monitor="val_loss", 
                                        mode="min", 
                                        patience=5, 
                                        restore_best_weights = True)

In [ ]:
%pip install keras==2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 566.4 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires libclang>=13.0.0, which is not installed.
tensorflow 2.9.1 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.9.1 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.8.0 which is incompatible.
tensorflow 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [28]:
# model = Sequential()
history = model.fit_generator(train_data,
                              verbose=1,
                              callbacks = [earlystopping],
                              epochs=12,
                              validation_data=(val_data))

/var/folders/h0/pjxkfbkd0jn4b1ngbzy6_d3c0000gn/T/ipykernel_7432/791066185.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_data,


Epoch 1/12
11/11 [==============================] - 40s 4s/step - loss: 0.0615 - accuracy: 0.9848
Epoch 2/12
11/11 [==============================] - 42s 4s/step - loss: 0.0467 - accuracy: 0.9903
Epoch 3/12
11/11 [==============================] - 44s 4s/step - loss: 0.0376 - accuracy: 0.9938
Epoch 4/12
11/11 [==============================] - 44s 4s/step - loss: 0.0401 - accuracy: 0.9918
Epoch 5/12
11/11 [==============================] - 45s 4s/step - loss: 0.0752 - accuracy: 0.9759
Epoch 6/12
11/11 [==============================] - 44s 4s/step - loss: 0.1602 - accuracy: 0.9439
Epoch 7/12
11/11 [==============================] - 43s 4s/step - loss: 0.1310 - accuracy: 0.9494
Epoch 8/12
11/11 [==============================] - 46s 4s/step - loss: 0.0786 - accuracy: 0.9774
Epoch 9/12
11/11 [==============================] - 44s 4s/step - loss: 0.0512 - accuracy: 0.9875
Epoch 10/12
11/11 [==============================] - 43s 4s/step - loss: 0.0343 - accuracy: 0.9918
Epoch 11/12
11/11 [